In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
import ppscore as pps
import statsmodels.api as sm
import matplotlib.pyplot as plt
import feature_engine
from datetime import timedelta
from datetime import datetime
from sklearn.impute import SimpleImputer
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import sklearn
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from feature_engine.imputation import MeanMedianImputer
import math
import joblib
import pickle






In [2]:
excel_file_path = 'budgetusd.xlsx'
sheet_name = "Expenses"
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
# Load the Excel file into a pandas DataFrame


In [3]:
# This function will remove decimal points

def round_up_numeric(value):
    if pd.notnull(value) and isinstance(value, (int, float)):
        return math.ceil(value * 100) / 100
    return value

df =  df.applymap(round_up_numeric)

In [4]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set 'Date' as the index
df.set_index('Date', inplace=True)


In [5]:
duplicate_cols = df.columns[df.columns.duplicated()]
print("Duplicate Columns:", duplicate_cols)

Duplicate Columns: Index([], dtype='object')


In [6]:
df.isnull().sum()

Resturant Revenue        0
Bar Revenue              0
Rooms Revenue            0
Total Revenue            0
Restaurant D.Cost        0
Bar D.Cost               0
Hotel D.Cost             0
Total D. Costs           0
Eventuals Wages          0
Uniforms                 0
Bonuses                  0
Social Security          0
Wages                    0
Staff Food               0
Total Wages              0
Maintenance              0
Electricity              0
Accounting Services      0
Water                    0
Gas                      0
Admin Expenses           0
Cable & Internet         0
Spotify                  0
Telefono                 0
Insuraces                0
Transport                0
 Booking.com             0
POS System               0
Google Mail              0
Marketing                0
Little Hotelier          0
Comisiones POS           0
Total Indirect Costs     0
Total Operating Costs    0
GOP                      0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 48 entries, 2022-01-01 to 2025-12-01
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Resturant Revenue      48 non-null     float64
 1   Bar Revenue            48 non-null     float64
 2   Rooms Revenue          48 non-null     float64
 3   Total Revenue          48 non-null     float64
 4   Restaurant D.Cost      48 non-null     float64
 5   Bar D.Cost             48 non-null     float64
 6   Hotel D.Cost           48 non-null     float64
 7   Total D. Costs         48 non-null     float64
 8   Eventuals Wages        48 non-null     float64
 9   Uniforms               48 non-null     float64
 10  Bonuses                48 non-null     float64
 11  Social Security        48 non-null     float64
 12  Wages                  48 non-null     float64
 13  Staff Food             48 non-null     float64
 14  Total Wages            48 non-null     f

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatter plots for numerical variables
sns.pairplot(df, vars=['Resturant Revenue', 'Bar Revenue', 'Rooms Revenue', 'Total Revenue', 'Restaurant D.Cost', 'Bar D.Cost', 'Hotel D.Cost',
'Total D. Costs', 'Eventuals Wages', 'Uniforms', 'Bonuses', 'Social Security', 'Wages', 'Staff Food', 'Total Wages', 'Maintenance',
'Electricity', 'Accounting Services', 'Water', 'Gas', 'Admin Expenses',	'Cable & Internet', 'Spotify',	'Telefono',	'Insuraces',
'Transport', 'Booking.com',	'POS System',	'Google Mail',	'Marketing',	'Little Hotelier',	'Comisiones POS',	'Total Indirect Costs',
'Total Operating Costs',	'GOP'])
plt.show()

# Heatmap for numerical variable correlations
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()



KeyError: 'Total D. Costs'

Error in callback <function flush_figures at 0x7f8075a33ba0> (for post_execute):


KeyboardInterrupt: 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 48 entries, 2022-01-01 to 2025-12-01
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Resturant Revenue      48 non-null     float64
 1   Bar Revenue            48 non-null     float64
 2   Rooms Revenue          48 non-null     float64
 3   Total Revenue          48 non-null     float64
 4   Restaurant D.Cost      48 non-null     float64
 5   Bar D.Cost             48 non-null     float64
 6   Hotel D.Cost           48 non-null     float64
 7   Total D. Costs         48 non-null     float64
 8   Eventuals Wages        48 non-null     float64
 9   Uniforms               48 non-null     float64
 10  Bonuses                48 non-null     float64
 11  Social Security        48 non-null     float64
 12  Wages                  48 non-null     float64
 13  Staff Food             48 non-null     float64
 14  Total Wages            48 non-null     f

In [14]:
df

,Resturant Revenue,Bar Revenue,Rooms Revenue,Total Revenue,Restaurant D.Cost,Bar D.Cost,Hotel D.Cost,Total D. Costs,Eventuals Wages,Uniforms,...,Transport,Booking.com,POS System,Google Mail,Marketing,Little Hotelier,Comisiones POS,Total Indirect Costs,Total Operating Costs,GOP
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,7771.64,3972.56,13136.89,24881.09,3696.59,1551.37,1471.00,6718.95,820.84,0.00,...,98.73,597.31,55.73,23.34,62.12,123.23,819.00,9304.80,16023.74,8857.35
2022-02-01,5660.00,3773.34,11430.53,20863.87,3476.78,2317.87,935.39,6730.03,864.03,0.00,...,98.73,117.67,55.73,23.34,62.12,123.23,519.45,8417.75,15147.78,5716.09
2022-03-01,5299.87,3533.23,9742.62,18575.70,2250.00,1944.45,1150.56,5345.00,555.14,89.73,...,209.14,627.39,56.03,30.20,99.31,118.00,370.31,9727.31,15072.31,3503.39
2022-04-01,11300.75,7533.84,15043.81,33878.39,4243.20,3128.95,2108.00,9480.14,1269.34,89.73,...,397.28,362.98,162.48,35.70,246.06,118.00,700.00,10911.03,20391.17,13487.23
2022-05-01,5555.01,3703.34,8546.17,17804.52,1666.67,1336.34,861.12,3864.12,661.16,0.00,...,157.17,363.99,56.19,35.80,144.93,118.00,500.00,9359.87,13223.98,4580.55
2022-06-01,2822.00,1881.33,7171.42,11874.74,1166.67,555.56,972.23,2694.45,227.09,0.00,...,161.50,543.12,56.19,35.80,118.46,118.00,305.56,8811.23,11505.68,369.07
2022-07-01,6345.99,4230.66,15864.98,26441.63,2611.12,1384.68,2219.18,6214.96,629.73,0.00,...,249.67,476.30,56.40,35.80,46.77,118.00,617.98,8953.12,15168.08,11273.55
2022-08-01,5398.01,3598.67,11925.83,20922.50,1861.12,1206.33,1249.32,4316.76,625.28,0.00,...,151.67,239.14,56.49,35.80,149.97,118.00,527.78,8614.86,12931.61,7990.89
2022-09-01,5495.22,3663.48,13738.03,22896.72,2138.89,1808.14,431.61,4378.63,487.64,0.00,...,248.06,263.17,56.49,35.80,170.25,118.00,515.03,10364.71,14743.34,8153.38
